In [1]:
import xml.etree.ElementTree as elemTree
import os
import sys
tree = elemTree.parse(r'../config/.config.xml')
root = tree.getroot()
xx = root.find('./PATHS')
work_path = xx.find('work').text
sys.path.append(work_path)

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm
from realtime_kiwoom.data_provider import *
import re

# TF로 부터 얻은 분봉 데이터

In [3]:
tree = elemTree.parse(r'../config/.config.xml')
root = tree.getroot()
node_sqlite3 = root.find('./DBMS/sqlite3')
config_db = {tag:node_sqlite3.find(tag).text for tag in ['database']}

In [4]:
config_db

{'database': 'db\\kiwoom_db.sqlite3'}

In [5]:
os.path.join(work_path, config_db['database'])

'c:\\Sungshin\\Lectures\\2022\\repos\\AIFT2022\\db\\kiwoom_db.sqlite3'

In [6]:
csv_paths = [ 
  ('069500', r'..\data\kodex_200.csv'),
  ('114800', r'..\data\kodex_inverse.csv'),
  ('226490', r'..\data\kodex_kospi.csv'),
  ('001', r'..\data\kospi.csv'),
  ('201', r'..\data\kospi200.csv')
]

In [7]:
provider_ = MinuteChartDataProvider(db_path=os.path.join(work_path, config_db['database']), table_name='data_in_minute', drop_table=False)

In [8]:
last_dt = provider_.query(f"SELECT dt FROM {provider_.table_name} WHERE st_code='069500' ORDER BY dt DESC LIMIT 1")

In [12]:
df = pd.read_csv(csv_paths[0][1], dtype={'체결시간':str})

In [ ]:
df.rename(columns={'체결시간':'dt'}, inplace=True)

In [19]:
f'dt > "{last_dt["dt"][0]}"'

'dt > "20220923153000"'

In [20]:
df.query(f'dt > "{last_dt["dt"][0]}"') if len(last_dt) > 0 else df

,현재가,거래량,dt,시가,고가,저가,수정주가구분,수정비율,대업종구분,소업종구분,종목정보,수정주가이벤트,전일종가


In [ ]:
minute_data_provider1 = MinuteChartDataProvider(db_path=os.path.join(work_path, config_db['database']), table_name='data_in_minute', drop_table=True)

In [ ]:
minute_data_provider1.query(f'select count(*) cnt from {minute_data_provider1.table_name}')

In [ ]:
# dfs = []
for st_code, csv_path in tqdm(csv_paths):
  df = pd.read_csv(csv_path, dtype={'체결시간':str})
  # df = minute_data_provider.filter_from_raw_data(df, st_code)
  minute_data_provider1.insert_by_dataframe(df, st_code)
  # dfs.append(df)
# whole_df = pd.concat(dfs, ignore_index=True)

In [ ]:
history_df = minute_data_provider1.query(f'select * from {minute_data_provider1.table_name} where dt > "20220101000000" order by dt asc')

In [ ]:
history_df

In [ ]:
history_df.query('st_code=="069500"').plot(x='dt', y='close')

# 주식 체결데이터로부터 분봉 데이터 생성

In [ ]:
sample_file_path = r'..\data\주식체결_sample.txt'
p = re.compile(r".*([{]'code.*[}])")
provider = RealTimeTickDataPrivder()

In [ ]:
with open(sample_file_path, 'r', encoding='utf8') as f:
  for line in f:
    m = p.match(line)
    if m:
      # print(eval(m.group(1)))
      provider.insert_by_query(eval(m.group(1)))

# 조회 쿼리

현재 삽임된 레코드 수 (틱)

In [ ]:
provider.query('SELECT count(*) cnt FROM today_in_ticks')

마지막 체결 시각 (틱)

In [ ]:
provider.query('select dt from today_in_ticks order by dt desc limit 1 ')

존재하는 테이블 조회

In [ ]:
provider.query('select name from sqlite_master where type="table"')

틱을 분봉으로 조회하는 쿼리

In [ ]:
today_df = provider.make_minute_chart_df()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.query("st_code=='069500'")['dt'],
                open=df.query("st_code=='069500'")['open'],
                high=df.query("st_code=='069500'")['high'],
                low=df.query("st_code=='069500'")['low'],
                close=df.query("st_code=='069500'")['close'])])
fig.show()

In [ ]:
history_df.query("st_code=='069500'")

In [ ]:
today_df.query('st_code=="069500"')

In [ ]:
df = pd.concat((history_df.query("st_code=='069500'"), today_df.query('st_code=="069500"')), ignore_index=True)

# 결합 테스트

스태틱 소스
- 어제까지 확보한 분봉 데이터 (TR 요청):: db/kiwoom_db.sqlite3:data_in_minute
- 오늘 장중에 확보한 분봉 데이터 (TR 요청):: db/kiwoom_db.sqlite3:today_in_minute
    - 장해 등으로 장중 (예: 11시)에 재접속 시 생성
        - TR요청으로 수집 

동적 소스
- 오늘 실시간 틱 데이터:: 인메모리 디비:today_in_ticks
- 오늘 틱 데이터로부터 생성한 분봉 데이터:: 인메모리 디비:today_in_minute 

결합
- `정적저장소`:data_in_minute + today_in_minute + `인메모리`:today_in_minute

In [ ]:
provider.query(f"select count(*) from {provider.table_name}")

In [ ]:
merged_df = pd.concat((history_df.query("st_code=='069500'"),
today_df.query('st_code=="069500"')), ignore_index=True)

In [ ]:
merged_df.head()

## 의의로...  SQLite3 -> Pandas 로딩후 dt를 Timestamp 변환시 시간 걸림

- 2000 행에 대해 요청시, 399ms 걸림

In [ ]:
pd.to_datetime(merged_df[-2000:]['dt'], format='%Y%m%d%H%M%S')

작전
- 장시작전: 어제까지의 분봉 데이터는 SQL -> pd.DataFrame 변환 해 둔다. (DataProvider 역할)
- 장중: 오늘의 분봉 데이터를 SQlite3 -> pd.DataFrame 변환한다. (DataProvider 역할)
- 합친다.